# Multi-Agent Bidding (Baselines)

In this walkthrough, we'll provide a brief example of how to use the custom bidding environment, as well as implementing some baselines. 

### Custom Environment Interaction

In [1]:
# imports
import gymnasium as gym
from torch.utils.tensorboard import SummaryWriter
import numpy as np

# stable baselines
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

# custom environment
from envs.bidding import BiddingEnv

env = BiddingEnv()

env.render(mode='verbose') # choose from verbose, bids, plot

Step: 0
Robot at (8, 5) with type navbot
Robot at (6, 0) with type navbot
Robot at (7, 1) with type humanbot
Robot at (8, 3) with type navbot
Robot at (2, 2) with type embedbot
Robot at (9, 7) with type embedbot
Robot at (3, 2) with type embedbot
Robot at (9, 1) with type navbot
Robot at (8, 4) with type humanbot

Task at (4, 1) with prize 3 and type transport
Task at (3, 7) with prize 2 and type specialty
Task at (1, 4) with prize 2 and type transport
Task at (9, 2) with prize 1 and type manipulation
Task at (9, 5) with prize 3 and type transport
Task at (9, 4) with prize 3 and type specialty

Bidding Matrix: 
╒══════════╤══════════╤══════════╤══════════╤══════════╤══════════╕
│   task 1 │   task 2 │   task 3 │   task 4 │   task 5 │   task 6 │
╞══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│        0 │        0 │        0 │        0 │        0 │        0 │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│        0 │        0 │        0 │       

In [6]:
obs = env.reset()

print('Observation space: \n', env.observation_space)
print('Action space: \n', env.action_space)
print('Random Action (Bid Matrix): \n', env.action_space.sample())

for step in range(10):
    
    obs, reward, done, truncated, info = env.step(env.action_space.sample())
    print(f'\n Step {step + 1} with reward = {reward}')

    if done:
        env.render(mode='verbose')
        print("Completed, final reward =", reward)
        break

Observation space: 
 Dict('robot_0': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)), 'robot_1': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)), 'robot_2': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)), 'robot_3': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)), 'robot_4': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)), 'robot_5': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)), 'robot_6': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)), 'robot_7': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)), 'robot_8': Dict('bidding_matrix': Box(0, 10, (9, 6), int8), 'self_state': Box(0, [10 10  2], (3,), int8)))
Action spac

### Benchmarking with Stable Baselines

In [7]:
env = BiddingEnv()

# check stable baselines compatibility
check_env(env, warn=True)

# tensorboard logging
tensorboard_log_dir = "./runs/bidding_stable_baselines"

/Users/devg/robopt/.venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:93: UserWarning: Nested observation spaces are not supported by Stable Baselines3 (Dict spaces inside Dict space). You should flatten it to have only one level of keys.For example, `dict(space1=dict(space2=Box(), space3=Box()), spaces4=Discrete())` is not supported but `dict(space2=Box(), spaces3=Box(), spaces4=Discrete())` is.
  warnings.warn(
/Users/devg/robopt/.venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
/Users/devg/robopt/.venv/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:464: UserWarning: Your action space has dtype int8, we recommend using np.float32 to avoid cast errors.
  warnings.warn(


In [23]:
# 1) baseline random policy
class RandomPolicy:
    def __init__(self, action_space):
        self.action_space = action_space

    def predict(self, observation):
        return self.action_space.sample()

# tensorboard random policy
log_dir = "./runs/baselines/random_policy"
writer = SummaryWriter(log_dir)

# evaluate random policy
random_policy = RandomPolicy(env.action_space)
obs, _ = env.reset(seed=42)
total_reward = 0

for _ in range(1000):  # 1000 steps for example
    action = random_policy.predict(obs)
    obs, reward, done, _, _ = env.step(action)
    total_reward += reward

    # log to tensorboard
    writer.add_scalar("Random_Policy/AvgBid", np.mean(action), step)
    writer.add_scalar("Random_Policy/StdBid", np.std(action), step)
    writer.add_scalar("Random_Policy/Reward", reward, step)

    if done:
        obs, _ = env.reset()

print(f"Total reward for random policy: {total_reward}")

writer.close() # close tensorboard writer

Total reward for random policy: -1546.5332292637909


In [19]:
# 2) baseline ppo policy
vec_env = make_vec_env(lambda: BiddingEnv(), n_envs=1) # vectorize env for ppo
ppo_model = PPO("MlpPolicy", vec_env, verbose=1, tensorboard_log=tensorboard_log_dir)
ppo_model.learn(total_timesteps=10000)

# evaluate ppo policy
mean_reward, std_reward = evaluate_policy(ppo_model, vec_env, n_eval_episodes=10)
print(f"Mean reward for PPO: {mean_reward}, Std: {std_reward}")

# close envs (unnecessary in exisitng close() implementation)
vec_env.close()

NotImplementedError: Nested observation spaces are not supported (Tuple/Dict space inside Tuple/Dict space).

In [29]:
# 3) baseline heuristic policy
class HeuristicPolicy:
    def __init__(self, bidding_matrix):
        self.bidding_matrix = bidding_matrix

    def predict(self, observation):
        # given prize range is (0, 4), always bid 2
        return np.ones(self.bidding_matrix.shape) * 1

# tensorboard random policy
log_dir = "./runs/baselines/heuristic_policy"
writer = SummaryWriter(log_dir)

# evaluate heuristic policy
heuristic_policy = HeuristicPolicy(env.bidding_matrix)
obs, _ = env.reset(seed=42)
total_reward = 0

for _ in range(1000):  # 1000 steps for example
    action = heuristic_policy.predict(obs)
    obs, reward, done, _, _ = env.step(action)
    total_reward += reward

    # log to tensorboard
    writer.add_scalar("Random_Policy/AvgBid", np.mean(action), step)
    writer.add_scalar("Random_Policy/StdBid", np.std(action), step)
    writer.add_scalar("Random_Policy/Reward", reward, step)

    if done:
        obs, _ = env.reset()

print(f"Total reward for heuristic policy: {total_reward}")

writer.close() # close tensorboard writer

Total reward for heuristic policy: -1777.2110281092396
